# Skyrim Statistics

### By Jules Alexandre

#### Skyrim is a game full of details with hundreads of NPCs and informations, Let's analyse and build some correlations over the game's data
#### We basically have :
- NPCs with their : Names, City of residence, Gender, Race, Occupation
- Cities with : Location, Weather, Who Controls it, how many inhabitants, etc.
